# Optimization using the ProblemInterface class

This notebook will show the basics of setting up a resilience optimization problem with the `Problem` class in `fmdtools.sim.search` module.

```
Copyright © 2024, United States Government, as represented by the Administrator of the National Aeronautics and Space Administration. All rights reserved.

The “"Fault Model Design tools - fmdtools version 2"” software is licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

In [1]:
import fmdtools.analyze as an
from pump_stochastic import Pump

ImportError: DLL load failed while importing _path: The specified module could not be found.

### Problem Setup
The `search` module can be used to define an optimization problem around and fmdtools model/simulation in terms of variables, objectives, and constraints.

Different classes enable the optimization of faults, disturbances, and parameters. Below we define a `DisturbanceProblem`, which will optimize the `s.eff` state in the `move_water` function at time `t=20`

In [2]:
from fmdtools.sim.search import DisturbanceProblem
mdl=Pump()
new_problem = DisturbanceProblem(mdl, 20, "move_water.r.s.eff", faultseq={14: {"import_water": "less_wat"}})

In this case, we are optimizing the total flow in the `MoveWater` function as read at time t=25. 

In [3]:
new_problem.add_result_objective("f1", "move_water.s.total_flow", time=25, negative=True)

We can additionally add constraints to the problem, in this case pressure as read at t=25.

In [4]:
new_problem.add_result_constraint('g1', "wat_1.s.pressure", time=25, threshold=20, comparator='less', negative=True)

Note that if all objectives and constraints are sampled in time before the defined simulation end-point, it will finish before completion to save computational time.


In [5]:
new_problem

The string representation of the problem shows how the objectives/constraints have been set up, that reflects:
- the form of the objectives as positive or negative for maximization/minimization (set with the `metric` parameter in `add_objective`).
- the form of the constraints as positive or negative based on the `threshold` parameter in `add_constraint` and overall `negative` kwargs.

These parameters will need to be adjusted depending on whether the interfacing optimization package is set up for minimization or maximization or in positive or negative null form (where feasible means positive or negative constraint values).

### Problem interfaces
Now that this problem is set up, we now have interfaces which can be passed to optimization methods. These are methods which *match the names of the objectives/constraints defined earlier* which can be passed as callables to optimizaiton methods.

In [6]:
new_problem.f1(1)
new_problem.f1(0.5)
new_problem.f1(1.5)
new_problem.f1(2)
new_problem.f1(0.3)

Note that despite being different callables, to reduce simulation costs, `obj_1` and `con_1` only *simulate the model* when a new variable value is entered into the problem. This can be seen by looking at the `current_iter` __repr__, which shows the values of the objectives/constraints a the current variable value.

In [7]:
new_problem

Additionally, we can look at the `iter_hist` iteration history for the problem.

In [8]:
fig, ax = new_problem.iter_hist.plot_line('objectives.f1', 'variables.move_water.r.s.eff',
                                          'constraints.g1')

As well as the actual history of the sim at the current variable value:

In [9]:
fig, ax = new_problem.hist.plot_line('fxns.move_water.s.eff', 'fxns.move_water.s.total_flow', 'flows.wat_1.s.flowrate', 'flows.wat_1.s.pressure',
                                     time_slice=[14, 20, 25])

### Optimization:
Now, we will demonstrate optimization using this problem as it is set up:

In [10]:
from scipy.optimize import minimize

In [11]:
con_list = [{'type': 'ineq', 'fun':getattr(new_problem, con)} for con in new_problem.constraints]

Note that scipy minimize assumes that:
- objectives are to be minimized, and
- constraints must be held positive 

The problem should thus be set up to accomodate this, by adjusting whether `negative=True` is sent to `add_result_objective` or `add_result_constraint`/

In [12]:
res = minimize(new_problem.f1, [1], constraints=con_list)

In [13]:
res

As shown, the variables are optimized to a value of x=4.0, the maximum possible value of MoveWater.eff which was put in the problem. We can further verify the optimized result by looking at the problem:

In [14]:
new_problem

As shown, the bound set constraint x>0 is active at the found minimum, as we would expect.

We can further visualize this solution using:

In [15]:
fig, ax = new_problem.iter_hist.plot_line('objectives.f1', 'variables.move_water.r.s.eff',
                                          'constraints.g1')

In [16]:
fig, ax = new_problem.hist.plot_line('fxns.move_water.s.eff', 'fxns.move_water.s.total_flow',
                                     'flows.wat_1.s.flowrate', 'flows.wat_1.s.pressure',
                                     time_slice=[14, 20, 25])

As shown, the constraint `g1` is active, meaning the optimal presure at `t=20` is just at the threshold of 20. This corresponds to the optimized `MoveWater.eff` value of 4.0.

In [17]:
assert abs(new_problem.constraints['g1'].value) < 0.001
assert abs(new_problem.variables['move_water.r.s.eff'] - 4.0)  < 0.001

### Multi-scenario Optimization

In addition to optimizing over single-scenarios `ParameterSimProblem` can be used to optimize over lists of scenarios from an `FaultSample`.

Here we define a slightly different problem, where instead of optimizing variable changes at specific times (e.g., faults), we instead optimize the model parameter `delay`, which changes how long of a delay there is before a fault when there is adverse pressure.

To see the effect of this accross scenarios, we first define a `FaultSample`:

In [18]:
from fmdtools.sim.sample import FaultDomain, FaultSample
fd = FaultDomain(mdl)
fd.add_all_fxn_modes("export_water")
fs = FaultSample(fd)
fs.add_fault_phases("on", args=(4,))

In [19]:
fs

We also need to define a `ParameterDomain` for the `delay` parameter to optimize:

In [20]:
from fmdtools.sim.sample import ParameterDomain
from examples.pump.pump_stochastic import PumpParam
pd = ParameterDomain(PumpParam)
pd.add_variables("delay")
pd(4)

We can now set up the `ParameterSimProblem`:

In [21]:
from fmdtools.sim.search import ParameterSimProblem
psp = ParameterSimProblem(mdl, pd, 'fault_sample', fs)

Our objective for this problem will be to minimize the cost model overall scenarios that is defined in `mdl.find_classification`.

In [22]:
psp.add_result_objective("f1", 'endclass.expected_cost')

We can then verify the problem setup. Note that no constraints will be used in this problem:

In [23]:
psp
psp.f1(15)

In [24]:
psp.f1(10)

In [25]:
fig, ax = psp.hist.plot_line('fxns.move_water.s.eff', 'fxns.move_water.s.total_flow',
                             'flows.wat_1.s.flowrate', 'flows.wat_1.s.pressure')

In [26]:
psp.res

We may now minimize the objective function:

In [27]:
res = minimize(psp.f1, [5], method="SLSQP", bounds=[[0,40]])

In [28]:
res

In [29]:
res = minimize(psp.f1, [5], method="nelder-mead")

In [30]:
res

Interestingly enough, while the optimizer gives a "optimization terminated successfully," it stays at the initial point. This may be because of a poor fit of oftimization method. See:

In [31]:
objs = [psp.f1(i) for i in range(100)]

In [32]:
from matplotlib import pyplot as plt
plt.plot(objs)
plt.scatter(res.x, res.fun)

As shown, the objective appears to be non-differentiable, with several plateaus between the starting point (20) and the minimum. Since the `SLSQP` solver is a gradient-based solver, it probably sees the gradient as 0 at this point, making it think the result is already an optimum. 

While many different optimization packages exist, one of the more-developed ones is the `pymoo` package ([see reference](https://pymoo.org/)). Below we show how to interface with `pymoo` to use a solver that will find the optimal solution.

In [33]:
# import pip
# pip.main(['install', 'pymoo'])


Below we set the problem up as a [pymoo problem object](https://pymoo.org/interface/problem.html) which can be used with a pymoo algorithm per the documentation. Note that this object corresponds directly (that is, is linked) to the original problem (see below).

In [34]:
from pymoo.core.problem import ElementwiseProblem
class Prob(ElementwiseProblem):
    def __init__(self, problem_inter):
        self.problem_inter = problem_inter
        super().__init__(n_var=1, n_obj=1, n_con=0.0, xl=0.0, xu=40)
    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = self.problem_inter.f1(x)

We can then optimize with a PatternSearch:

In [35]:
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch
import numpy as np

In [36]:
algorithm=PatternSearch(x0=np.array([5])) 

In [37]:
pymoo_prob = Prob(psp)

In [38]:
res = minimize(pymoo_prob, algorithm, verbose=True)

In [39]:
res.X

In [40]:
res.F

In [41]:
plt.plot(objs)
plt.scatter(res.X, res.F)

As shown, this method more capably finds the minimum in this case, in part because the underlying search algorithm (Hooke and Jeeves Pattern Search) is more robust to this type of problem.

We can visualize the results of this problem by looking at the simulation log:

In [42]:
fig, ax = psp.iter_hist.plot_line('objectives.f1', 'variables.delay')

As well as the history:

In [43]:
fig, ax = psp.hist.plot_line('fxns.move_water.s.eff', 'fxns.move_water.s.total_flow',
                             'flows.wat_1.s.flowrate', 'flows.wat_1.s.pressure')

As shown, the main difference is that at the initial point, the short delay causes the pump to break during the time of the simulation, while at a long delay, the pump breaks later in the simulation. The optimum is at t=40 since this is the first feasible point where three scenarios no longer result in a mechanical break of the pump.